## imports

In [10]:
from models.test_model import TestModel

from langchain_core.prompts import ChatPromptTemplate

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from tools.prisoners_dilemma_tools import PrisonersDilemmaTools

tools = [TavilySearchResults(max_results=1)]

In [3]:
# LLM-based personality
personality_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an agent in a Prisoner's Dilemma game. Your personality is defined below:"),
    ("system", "Personality: {personality}"),
    ("user", "Craft a message to the other agent, based on your personality, to convince them whether you will cooperate or deceive.")
])

def create_personality_agent(personality):
    # Use the LLM to initialize agent personality and behavior
    return {
        "personality": personality,
        "intent": "",
        "received_message": ""
    }


In [4]:
def craft_message_with_llm(agent):
    # Construct the message crafting prompt for the LLM
    prompt = f"""
    You are playing the Prisoner's Dilemma. 
    Your personality is as follows: {agent['personality']}
    Do you want to cooperate or deceive? Compose a message to the other player that reflects your intent.
    """
    
    # Generate the message via the LLM
    response = llm.invoke({"messages": [prompt]})
    agent['intent'] = response['content']
    return response['content']


In [5]:
def message_router(agent1_message, agent2_message):
    # Append messages to each other's states
    agent1["received_message"] = agent2_message
    agent2["received_message"] = agent1_message


In [6]:
def make_decision_with_llm(agent):
    # Construct the decision prompt based on received message
    decision_prompt = f"""
    You are playing the Prisoner's Dilemma. 
    Your personality is as follows: {agent['personality']}
    The other agent told you: {agent['received_message']}
    You have said that you will: {agent['intent']}. 
    Do you decide to cooperate or betray? Respond with 'cooperate' or 'betray'.
    """
    
    # Generate the final decision via LLM
    response = llm.invoke({"messages": [decision_prompt]})
    return response['content']


In [12]:
from langgraph import GraphNode, LangGraph

# Define nodes for the graph
message_node_agent1 = GraphNode(name="Agent1 Message", func=craft_message_with_llm, input_key="agent1")
message_node_agent2 = GraphNode(name="Agent2 Message", func=craft_message_with_llm, input_key="agent2")
router_node = GraphNode(name="Message Router", func=message_router)
decision_node_agent1 = GraphNode(name="Agent1 Decision", func=make_decision_with_llm, input_key="agent1")
decision_node_agent2 = GraphNode(name="Agent2 Decision", func=make_decision_with_llm, input_key="agent2")

# Construct the graph
graph = LangGraph(
    nodes=[message_node_agent1, message_node_agent2, router_node, decision_node_agent1, decision_node_agent2],
    edges=[
        ("Agent1 Message", "Message Router"),
        ("Agent2 Message", "Message Router"),
        ("Message Router", "Agent1 Decision"),
        ("Message Router", "Agent2 Decision"),
    ]
)

llm = TestModel()

# Initialize agents
agent1 = create_personality_agent("Rational and cautious. Values long-term trust.")
agent2 = create_personality_agent("Opportunistic, but cooperative when necessary.")

# Run the graph
result = graph.run({"agent1": agent1, "agent2": agent2})

# Output the decisions
agent1_decision = result['agent1_decision']
agent2_decision = result['agent2_decision']

print(f"Agent 1 decided to: {agent1_decision}")
print(f"Agent 2 decided to: {agent2_decision}")


ImportError: cannot import name 'GraphNode' from 'langgraph' (unknown location)